# Export file to Bit-Bucket Using Job Parameters
- This notebook exports a data asset (saved in a deployment space) to bit-bucket server.
- Run this notebook in a CPD cluster as a job with the parameters given below.

# Params
- Please provide the following as the job environment variables.
    - `BITBUCKET_TOKEN=xxxx-xxx-xxxx-xxx`, personal/service access token from bit-bucket
    - `BITBUCKET_URL=SERVER_URL/rest/api/1.0`, use `SERVER_URL` of the bit-bucket server. Be sure to add the `/rest/api/1.0` at the end. 
    - `BITBUCKET_PROJECT=test_project_name`, name of the bit-bucket project where the file needs to be exported 
    - `BITBUCKET_REPOSITORY=test_repository_name`, name of the bit-bucket repository where the file needs to be exported 
    - `BITBUCKET_BRANCH=test_branch_name`, name of the bit-bucket branch from where the file needs to be exported 
    - `BITBUCKET_FILE_NAME=test_file_name`, name of the file to be exported to the bit bucket server
    - `DEPLOYMENT_SPACE_ID=yyyy-yyyy-yyy-yyy-yyyy`, analytic deployment space id. You can find it from the settings tab of the space.

### NOTE: The environment variable values must not have any spaces as well as not encapsulated in quotes. Also, no space between the variable name and `=` and its value.

# How to Run
1. Open this notebook in a CPD project and save a version of it using `File->Save Version`.
2. If running for the first time or if want to create a new job:
    1. Select the `Jobs` button on the top right corner (next to the share button) and select `Create a Job`. This will open a new browser tab to create a job. 
    2. On the `Create a Job` browser tab:
        1. Give a unique job name.
        2. Select `Configure Additional Settings` and provide the parametes mentioned above with their values.
        3. Select `Create and Run` button on the bottom right.
3. If want to rerun the notebook as an existing job:
    1. Select the `Jobs` button on the top right corner (next to the share button) and select `Save and view jobs`. 
    2. You will be provided with the names of existing jobs. 
    3. Select the `Edit` link for the `Environment Variables` tab.
    4. Select the `Play` button on the top right to run the job again with the parameters provided.

In [22]:
!pip install numpy==1.16.6

In [23]:
!pip install watson-machine-learning-client-V4

In [27]:
# common imports
import os
import requests
import ntpath
import base64
import tarfile
from requests.auth import HTTPBasicAuth
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [28]:
# # save the info being passed in as environment variable parameters
# bb_token = os.environ.get('BITBUCKET_TOKEN')
# bb_url = os.environ.get('BITBUCKET_URL')
# bb_project = os.environ.get('BITBUCKET_PROJECT')
# bb_repo = os.environ.get('BITBUCKET_REPOSITORY')
# bb_branch = os.environ.get('BITBUCKET_BRANCH')
# file_name = os.environ.get('BITBUCKET_FILE_NAME')
# file_commit_message = os.environ.get('BITBUCKET_FILE_COMMIT_MESSAGE')
# file_source_commit_id = os.environ.get('BITBUCKET_FILE_SOURCE_COMMIT_ID')
# space_id = os.environ.get('DEPLOYMENT_SPACE_ID')
# data_asset_id = os.environ.get('DATA_ASSET_ID')

In [51]:
# debug
space_id = "135c8a07-1fc0-4ae7-a03f-19b27cfd48ee"
data_asset_id = "fbfa279b-f13e-4d4a-9196-c647f4f30343"

bb_token="NTExNzg4MDc2MTkzOuU+9qUSBQ6/iS1mO6xesYpmMSSj"
bb_url="http://localhost:7990/rest/api/1.0"
#url="http://localhost:7990"
bb_project="cpd"
bb_repo="cpd_int_test_3"
file_name="js_training_distribution.json"
bb_branch="test_branch"
file_commit_message = "Test Commit Message"
file_source_commit_id = "null"

In [41]:
# # instantiate the wml service running locally on this cpd cluster
# # Use "token": os.environ['USER_ACCESS_TOKEN'] as an alternative to username and password
# wml_credentials = {
#    "token": os.environ['USER_ACCESS_TOKEN'],
#    "instance_id" : "wml_local",
#    "url": os.environ['RUNTIME_ENV_APSX_URL'],
#    "version": "3.0.0"
# }
# wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [42]:
# instantiate the wml service running locally on this cpd cluster
# Use "token": os.environ['USER_ACCESS_TOKEN'] as an alternative to username and password
wml_credentials={
         "url": "https://namespace1-cpd-namespace1.apps.ylidev.demo.ibmcloud.com/",
        "username": "jassingh",
        "password": "ibm123",
        "instance_id": "wml_local",
        "version": "3.0.0"
}

wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [43]:
# set the default deployment space id passed in.
wml_client.set.default_space(space_id)

'SUCCESS'

In [44]:
# download the latest file to the local file system
wml_client.data_assets.download(data_asset_id,file_name);

Successfully saved data asset content to file: 'js_training_distribution.json'


In [45]:
# get the file contents
f = open(os.path.abspath(file_name), "rb+")
contents = f.read()

In [46]:
# display contents
contents

b'{\n  "common_configuration": {\n    "problem_type": "binary",\n    "label_column": "score",\n    "input_data_schema": {\n      "type": "struct",\n      "fields": [\n        {\n          "name": "Order",\n          "type": "long",\n          "nullable": true,\n          "metadata": {\n            "modeling_role": "feature"\n          }\n        },\n        {\n          "name": "Property Id",\n          "type": "long",\n          "nullable": true,\n          "metadata": {\n            "modeling_role": "feature"\n          }\n        },\n        {\n          "name": "DOF Gross Floor Area",\n          "type": "double",\n          "nullable": true,\n          "metadata": {\n            "modeling_role": "feature"\n          }\n        },\n        {\n          "name": "Year Built",\n          "type": "long",\n          "nullable": true,\n          "metadata": {\n            "modeling_role": "feature"\n          }\n        },\n        {\n          "name": "Number of Buildings - Self-reported

In [47]:
# prepare the contents to be pushed over to the server
if file_source_commit_id == "null":
    input_json={
        'content':contents,
        'message':file_commit_message,
        'branch':bb_branch
    }
else:
    input_json={
        'content':contents,
        'message':file_commit_message,
        'branch':bb_branch,
        'sourceCommitId':file_source_commit_id
    }

In [48]:
# build the url to be used with http request to get the raw contents of the file 
#fileurl='{0}/projects/{1}/repos/{2}/raw/{3}?at={4}'.format(bb_url,bb_project,bb_repo,file_name,bb_branch)

# build the url to be used with http request to push the file over 
push_file_url='{0}/projects/{1}/repos/{2}/browse/{3}?at={4}'.format(bb_url,bb_project,bb_repo,file_name,bb_branch)
print (push_file_url)

http://localhost:7990/rest/api/1.0/projects/cpd/repos/cpd_int_test_3/browse/js_training_distribution.json?at=test_branch


In [49]:
# call the http get request to get the file using the bit-bucket token provided by the user.
#response = requests.get(fileurl,headers={'Authorization': 'Bearer ' + bb_token}, verify=False)
response = requests.put(push_file_url,headers={'Authorization': 'Bearer ' + bb_token}, files={'':''}, data=input_json, verify=False)

In [50]:
# print the put request details
response

<Response [200]>